## SMIB

Synchronous Machine Infinite Bus (SMIB) system


### System building

In [1]:
from pydae.bmapu.bmapu_builder import  bmapu

from pydae.build_v2 import builder
import sympy as sym
import numpy as np

#### System data

In [2]:
data = {
"system":{"name":"smib","S_base":100e6, "K_p_agc":0.0,"K_i_agc":0.0,"K_xif":0.01},       
"buses":[{"name":"1", "P_W":0.0,"Q_var":0.0,"U_kV":20.0},
         {"name":"2", "P_W":0.0,"Q_var":0.0,"U_kV":20.0}
        ],
"lines":[{"bus_j":"1", "bus_k":"2", "X_pu":0.05,"R_pu":0.01,"Bs_pu":1e-6,"S_mva":100.0}],
"syns":[
      {"bus":"1","S_n":100e6,
         "X_d":1.8,"X1d":0.3, "T1d0":8.0,    
         "X_q":1.7,"X1q":0.55,"T1q0":0.4,  
         "R_a":0.01,"X_l": 0.2, 
         "H":5.0,"D":1.0,
         "Omega_b":314.1592653589793,"omega_s":1.0,"K_sec":0.0,
         "avr":{"type":"sexs","K_a":100.0,"T_a":0.1,"T_b":1.0,"T_e":0.1,"E_min":-10.0,"E_max":10.0,"v_ref":1.0},
         "K_delta":0.0}],
"genapes":[{"bus":"2","S_n":1e9,"F_n":50.0,"X_v":0.001,"R_v":0.0,"K_delta":0.001,"K_alpha":1e-6}]
}

#### System generation and compilation

In [3]:
grid = bmapu(data)
grid.uz_jacs = True
grid.construct('smib')

b = builder(grid.sys_dict)
b.dict2system()
b.functions()
b.jacobians()
b.cwrite()
b.compile()
b.template()

In [4]:
import smib

In [5]:
model = smib.model()
model.ini({},'xy_0.json')
model.report_y()

V_1   =  1.00
theta_1 =  0.05
V_2   =  1.00
theta_2 =  0.00
i_d_1 =  0.84
i_q_1 =  0.55
p_g_1 =  0.99
q_g_1 = -0.17
v_f_1 =  1.92
omega_2 =  1.00
i_d_2 =  0.02
i_q_2 = -0.10
p_s_2 = -0.10
q_s_2 =  0.02
omega_coi =  1.00
p_agc =  0.00


In [6]:
data = np.ones(len(grid.dae['f']))
x = np.ones(len(grid.dae['x']))
y = np.ones(len(grid.dae['y_ini']))
u = np.ones(len(grid.dae['u_ini_dict']))
p = np.ones(len(grid.dae['params_dict']))


In [7]:
from cffi import FFI

ffi = FFI()

pdata = ffi.cast("double *", data.ctypes.data)
px = ffi.cast("double *", x.ctypes.data)
py = ffi.cast("double *", y.ctypes.data)
pu = ffi.cast("double *", u.ctypes.data)
pp = ffi.cast("double *", p.ctypes.data)

def evaluate_cffi(pdata,px,py,pu,pp,Dt):

    for it in range(10):
        smib_cffi.lib.f_eval(pdata,px,py,pu,pp,Dt)



In [8]:
Dt = 0.001
%timeit evaluate_cffi(pdata,px,py,pu,pp,Dt)

AttributeError: cffi library 'smib_cffi' has no function, constant or global variable named 'f_eval'

In [14]:
data

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [15]:
smib_cffi.ffi.bit_generator

AttributeError: '_cffi_backend.FFI' object has no attribute 'bit_generator'

In [16]:
import numba
import numba.core.typing.cffi_utils as cffi_support
import cffi

ffi = cffi.FFI()


import smib_cffi
    
cffi_support.register_module(smib_cffi)
f_ini_eval = smib_cffi.lib.f_ini_eval



@numba.njit()
def evaluate(data,x,y,u,p,Dt):

    data_ptr=ffi.from_buffer(np.ascontiguousarray(data))
    x_ptr=ffi.from_buffer(np.ascontiguousarray(x))
    y_ptr=ffi.from_buffer(np.ascontiguousarray(y))
    u_ptr=ffi.from_buffer(np.ascontiguousarray(u))
    p_ptr=ffi.from_buffer(np.ascontiguousarray(p))

    f_eval(data_ptr,x_ptr,y_ptr,u_ptr,p_ptr,Dt)

@numba.njit()
def evaluate(data,x,y,u,p,Dt):

    data_ptr=ffi.from_buffer(np.ascontiguousarray(data))
    x_ptr=ffi.from_buffer(np.ascontiguousarray(x))
    y_ptr=ffi.from_buffer(np.ascontiguousarray(y))
    u_ptr=ffi.from_buffer(np.ascontiguousarray(u))
    p_ptr=ffi.from_buffer(np.ascontiguousarray(p))

    for it in range(10):

        f_ini_eval(data_ptr,x_ptr,y_ptr,u_ptr,p_ptr,Dt)

evaluate(data,x,y,u,p,Dt)

2022-12-28 02:13:36,718 bytecode dump:
>          0	NOP(arg=None, lineno=29)
           2	LOAD_GLOBAL(arg=0, lineno=29)
           4	LOAD_METHOD(arg=1, lineno=29)
           6	LOAD_GLOBAL(arg=2, lineno=29)
           8	LOAD_METHOD(arg=3, lineno=29)
          10	LOAD_FAST(arg=0, lineno=29)
          12	CALL_METHOD(arg=1, lineno=29)
          14	CALL_METHOD(arg=1, lineno=29)
          16	STORE_FAST(arg=6, lineno=29)
          18	LOAD_GLOBAL(arg=0, lineno=30)
          20	LOAD_METHOD(arg=1, lineno=30)
          22	LOAD_GLOBAL(arg=2, lineno=30)
          24	LOAD_METHOD(arg=3, lineno=30)
          26	LOAD_FAST(arg=1, lineno=30)
          28	CALL_METHOD(arg=1, lineno=30)
          30	CALL_METHOD(arg=1, lineno=30)
          32	STORE_FAST(arg=7, lineno=30)
          34	LOAD_GLOBAL(arg=0, lineno=31)
          36	LOAD_METHOD(arg=1, lineno=31)
          38	LOAD_GLOBAL(arg=2, lineno=31)
          40	LOAD_METHOD(arg=3, lineno=31)
          42	LOAD_FAST(arg=2, lineno=31)
          44	CALL_METHOD(arg

In [17]:
data = np.ones(len(grid.dae['f']))

%timeit evaluate(data,x,y,u,p,Dt)

584 ns ± 10.5 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [68]:
data

array([-1., -1.,  0., -1.,  1.,  1.,  0.,  0.,  0.,  0., -1.])